# Detailed tables

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "\\toprule\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"

    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()


def main(type_):
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        caption = f"Detailed PRR scores (raw and detrended) for all methods. Metric: {metric_name}."
        header = get_header(caption)
        header += "&" + "&".join([colname(dataset).replace('_', '\\_') for dataset in datasets]) + "\\\\\n"
        latex = header

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"
                latex += "\midrule\n"
                latex += "& \\multicolumn{8}{c}{" + model_title + "}\\\\\n"
                latex += "\midrule\n"

                ue_scores, _, _ = detrend_ue(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                    sorted_scores = sorted(scores, reverse=True)
                    best_map[col_idx] = sorted_scores[0]
                    second_map[col_idx] = sorted_scores[1]

                for method_full, method_short in methods_dict.items():
                    raw_scores = ue_scores[f"{method_short}_raw"]
                    detr_scores = ue_scores[f"{method_short}_detr"]

                    raw_row = [f"Raw {method_short}"]
                    detr_row = [f"Detr {method_short}"]

                    for col_idx, (raw, detr) in enumerate(zip(raw_scores, detr_scores)):
                        # Format raw
                        if np.isclose(raw, best_map[col_idx]):
                            raw_row.append(f"\\textbf{{{raw:.2f}}}")
                        elif np.isclose(raw, second_map[col_idx]):
                            raw_row.append(f"\\underline{{{raw:.2f}}}")
                        else:
                            raw_row.append(f"{raw:.2f}")

                        # Format detr
                        if np.isclose(detr, best_map[col_idx]):
                            detr_row.append(f"\\textbf{{{detr:.2f}}}")
                        elif np.isclose(detr, second_map[col_idx]):
                            detr_row.append(f"\\underline{{{detr:.2f}}}")
                        else:
                            detr_row.append(f"{detr:.2f}")

                    latex += " & ".join(raw_row) + " \\\\\n"
                    latex += " & ".join(detr_row) + " \\\\\n"
                    latex += "\\midrule\n"

        latex += footer()
        name = f'tables/{metric}_detailed_results_per_dataset_ranking.tex'
        with open(name, 'w') as f:
            f.write(latex)


if __name__ == "__main__":
    type_='all'
    main(type_)

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f52718e0e50>>
Traceback (most recent call last):
  File "/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrup

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f17ac0>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f017b0>]
llama wmt14_csen Below q ids: 1771


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f48e80>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4fd0182ef0>]
llama wmt14_deen Below q ids: 1796


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Accuracy':['gsm8k'],
    'AlignScoreInputOutput':['xsum']
}

METRICS = {
    'Accuracy': 'Accuracy',
    'AlignScoreInputOutput': 'Align Score',
    # 'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lc}\n"
                "\\toprule\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()


def main(type_):
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        caption = f"Detailed PRR scores (raw and detrended) for all methods. Metric: {metric_name}."
        header = get_header(caption)
        header += "&" + "&".join([colname(dataset).replace('_', '\\_') for dataset in datasets]) + "\\\\\n"
        latex = header

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"
                latex += "\midrule\n"
                latex += "& \\multicolumn{8}{c}{" + model_title + "}\\\\\n"
                latex += "\midrule\n"

                ue_scores, _ , _, _,_= detrend_ue_w_quality(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                    sorted_scores = sorted(scores, reverse=True)
                    best_map[col_idx] = sorted_scores[0]
                    second_map[col_idx] = sorted_scores[1]

                for method_full, method_short in methods_dict.items():
                    raw_scores = ue_scores[f"{method_short}_raw"]
                    detr_scores = ue_scores[f"{method_short}_detr"]

                    raw_row = [f"Raw {method_short}"]
                    detr_row = [f"Detr {method_short}"]

                    for col_idx, (raw, detr) in enumerate(zip(raw_scores, detr_scores)):
                        # Format raw
                        if np.isclose(raw, best_map[col_idx]):
                            raw_row.append(f"\\textbf{{{raw:.2f}}}")
                        elif np.isclose(raw, second_map[col_idx]):
                            raw_row.append(f"\\underline{{{raw:.2f}}}")
                        else:
                            raw_row.append(f"{raw:.2f}")

                        # Format detr
                        if np.isclose(detr, best_map[col_idx]):
                            detr_row.append(f"\\textbf{{{detr:.2f}}}")
                        elif np.isclose(detr, second_map[col_idx]):
                            detr_row.append(f"\\underline{{{detr:.2f}}}")
                        else:
                            detr_row.append(f"{detr:.2f}")

                    latex += " & ".join(raw_row) + " \\\\\n"
                    latex += " & ".join(detr_row) + " \\\\\n"
                    latex += "\\midrule\n"

        latex += footer()
        name = f'tables/{metric}_detailed_results_per_dataset_ranking_w_quality.tex'
        with open(name, 'w') as f:
            f.write(latex)


if __name__ == "__main__":
    type_='all'
    main(type_)

# Best scores

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()
def format_score_pair(raw, detr, threshold=0.00):
    arrow = "$\\uparrow$" if detr - raw > threshold else ""
    return f"{raw:.2f}", f"{detr:.2f}{arrow}"

def main(type_):
    caption = "Best raw and detrended PRR scores across all metrics and models."
    latex = "\\begin{table*}[ht]\n\\centering\n\\small\n"
    latex += f"\\caption{{{caption}}}\n"
    total_cols = max(len(v) for v in DATASETS.values())
    col_spec = "l" + "cc" * total_cols
    latex += f"\\begin{{tabular}}{{{col_spec}}}\n\\toprule\n"

    # Dataset header (generic, since all blocks will have same layout)
    dataset_line = "Model"
    subheader_line = " "
    example_metric = next(iter(METRICS))
    for d in DATASETS[example_metric]:
        name = colname(d)
        dataset_line += f" & \\multicolumn{{2}}{{c}}{{{name}}}"
        subheader_line += " & Raw & Detr"
    latex += dataset_line + " \\\\\n"
    latex += subheader_line + " \\\\\n\\midrule\n"

    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        # Section header for metric
        n_cols = 1 + 2 * len(datasets)
        latex += f"\\multicolumn{{{n_cols}}}{{c}}{{\\textbf{{{metric_name}}}}} \\\\\n"

        for model_key, model_name in MODELS.items():
            ue_scores, _, _ = detrend_ue(
                datasets, model_key, "base", [metric], ue_methods, methods_dict, return_unprocessed=True
            )

            row = [model_name]
            for i in range(len(datasets)):
                best_raw = -float("inf")
                best_detr = -float("inf")

                for method in ue_methods:
                    raw = ue_scores[f"{method}_raw"][i]
                    detr = ue_scores[f"{method}_detr"][i]
                    best_raw = max(best_raw, raw)
                    best_detr = max(best_detr, detr)

                raw_fmt, detr_fmt = format_score_pair(best_raw, best_detr)
                row.extend([raw_fmt, detr_fmt])

            latex += " & ".join(row) + " \\\\\n"

        latex += "\\midrule\n"

    latex += "\\bottomrule\n\\end{tabular}\n\\end{table*}\n"

    with open("tables/final_rowwise_metric_blocks.tex", "w") as f:
        f.write(latex)



if __name__ == "__main__":
    type_='all'
    main(type_)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f41cce629e0>]
llama wmt19_ruen Below q ids: 1752


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e6075e0>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e61a170>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f41cce63910>]
eurollm wmt19_ruen Below q ids: 1762


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


# Avg Improvements

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality, detrend_ue_w_quality_only
from pathlib import Path
import pathlib
import argparse
from sklearn.linear_model import LinearRegression

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'AlignScoreInputOutput' : ['xsum'],
        'Accuracy' : ['gsm8k'],
            # 'Rouge_rougeL' : ['xsum'],


}

METRICS = {
                # 'Rouge_rougeL' : 'Rouge L',
    # 'AlignScoreInputOutput': 'Align Score',
    'XComet-XCOMET-XXL': 'XComet XXL',
    # 'Accuracy':'Acc',
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX-XXL',
    'Comet-wmt22-comet-da' :'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        "\\toprule\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()

def plot_raw_vs_detrended(lengths, raw_scores, detr_scores, method_name, save_path=None):
    lengths = np.array(lengths).reshape(-1, 1)
    raw_scores = np.array(raw_scores)
    detr_scores = np.array(detr_scores)

    # Fit linear regression models
    reg_raw = LinearRegression().fit(lengths, raw_scores)
    reg_detr = LinearRegression().fit(lengths, detr_scores)

    # Predict
    raw_fit = reg_raw.predict(lengths)
    detr_fit = reg_detr.predict(lengths)

    plt.figure(figsize=(8, 6))
    plt.scatter(lengths, raw_scores, label='Raw Scores', alpha=0.6, color='blue')
    plt.plot(lengths, raw_fit, color='blue', linestyle='--', label=f'Raw Fit (slope={reg_raw.coef_[0]:.3f})')

    plt.scatter(lengths, detr_scores, label='Detrended Scores', alpha=0.6, color='green')
    plt.plot(lengths, detr_fit, color='green', linestyle='--', label=f'Detrended Fit (slope={reg_detr.coef_[0]:.3f})')

    plt.title(f'Length Effect for {method_name}')
    plt.xlabel('Sequence Length')
    plt.ylabel('Uncertainty Score')
    plt.legend()
    plt.grid(True)

    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    else:
        plt.show()


def main(type_):
    results = []
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"

                ue_scores, _, _ = detrend_ue(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                        # print( method_short, metric,datasets[col_idx], ':', raw, ' -> ', detr)
                        results.append({
                            "Metric": metric_name,
                            "Dataset": datasets[col_idx],
                            "Model": model_title,
                            "Method": method_short,
                            "ScoreType": "Raw",
                            "Score": raw
                        })

                        results.append({
                            "Metric": metric_name,
                            "Dataset": datasets[col_idx],
                            "Model": model_title,
                            "Method": method_short,
                            "ScoreType": "Detrended",
                            "Score": detr
                        })
    df = pd.DataFrame(results)
    csv_path = "ue_scores_results_wmt3.csv"
    df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")

                    # sorted_scores = sorted(scores, reverse=True)
                    # best_map[col_idx] = sorted_scores[0]
                    # second_map[col_idx] = sorted_scores[1]

    


if __name__ == "__main__":
    type_='all'
    main(type_)

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4265e83c10>]
gemma wmt19_lten Below q ids: 1800


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e51e770>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256c404f0>]
gemma wmt19_ruen Below q ids: 1764


...

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256764fa0>]
eurollm wmt19_lten Below q ids: 1783


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256aaba00>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426d8a81f0>]
eurollm wmt19_ruen Below q ids: 1762
Results saved to ue_scores_results_wmt3.csv


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality, detrend_ue_w_quality_only
from pathlib import Path
import pathlib
import argparse
from sklearn.linear_model import LinearRegression

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    # 'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'AlignScoreInputOutput' : ['xsum'],
        'Accuracy' : ['gsm8k'],
            # 'Rouge_rougeL' : ['xsum'],


}

METRICS = {
                # 'Rouge_rougeL' : 'Rouge L',
    'AlignScoreInputOutput': 'Align Score',
    # 'XComet-XCOMET-XXL': 'XComet XXL',
    'Accuracy':'Acc',
    # 'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX-XXL',
    # 'Comet-wmt22-comet-da' :'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        "\\toprule\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()

def plot_raw_vs_detrended(lengths, raw_scores, detr_scores, method_name, save_path=None):
    lengths = np.array(lengths).reshape(-1, 1)
    raw_scores = np.array(raw_scores)
    detr_scores = np.array(detr_scores)

    # Fit linear regression models
    reg_raw = LinearRegression().fit(lengths, raw_scores)
    reg_detr = LinearRegression().fit(lengths, detr_scores)

    # Predict
    raw_fit = reg_raw.predict(lengths)
    detr_fit = reg_detr.predict(lengths)

    plt.figure(figsize=(8, 6))
    plt.scatter(lengths, raw_scores, label='Raw Scores', alpha=0.6, color='blue')
    plt.plot(lengths, raw_fit, color='blue', linestyle='--', label=f'Raw Fit (slope={reg_raw.coef_[0]:.3f})')

    plt.scatter(lengths, detr_scores, label='Detrended Scores', alpha=0.6, color='green')
    plt.plot(lengths, detr_fit, color='green', linestyle='--', label=f'Detrended Fit (slope={reg_detr.coef_[0]:.3f})')

    plt.title(f'Length Effect for {method_name}')
    plt.xlabel('Sequence Length')
    plt.ylabel('Uncertainty Score')
    plt.legend()
    plt.grid(True)

    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    else:
        plt.show()


def main(type_):
    results = []
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"

                ue_scores, _, _,_,_ = detrend_ue_w_quality(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                        # print( method_short, metric,datasets[col_idx], ':', raw, ' -> ', detr)
                        results.append({
                            "Metric": metric_name,
                            "Dataset": datasets[col_idx],
                            "Model": model_title,
                            "Method": method_short,
                            "ScoreType": "Raw",
                            "Score": raw
                        })

                        results.append({
                            "Metric": metric_name,
                            "Dataset": datasets[col_idx],
                            "Model": model_title,
                            "Method": method_short,
                            "ScoreType": "Detrended",
                            "Score": detr
                        })
    df = pd.DataFrame(results)
    csv_path = "ue_scores_results_gsm8k_xsum.csv"
    df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")

                    # sorted_scores = sorted(scores, reverse=True)
                    # best_map[col_idx] = sorted_scores[0]
                    # second_map[col_idx] = sorted_scores[1]

    


if __name__ == "__main__":
    type_='all'
    main(type_)

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4265548670>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256d1bf70>]
llama xsum Below q ids: 1620


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4265b27820>]
llama gsm8k Below q ids: 1796


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256764e20>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4256aaafe0>]
gemma gsm8k Below q ids: 1798
Results saved to ue_scores_results_gsm8k_xsum.csv


In [24]:
import pandas as pd

# Load your actual DataFrame here
df = pd.read_csv("ue_scores_results_wmt3.csv")  # Or use your in-memory DataFrame
df_2 = pd.read_csv("ue_scores_results_gsm8k_xsum.csv")

df = pd.concat([df, df_2]).reset_index(drop=True)

# df = df[df['Metric']!='BLEU'].reset_index(drop=True)
# Calculate improvement per Method, Metric, Dataset
pivot = df.pivot_table(index=["Method", "Metric", "Dataset","Model"], columns="ScoreType", values="Score").reset_index()
pivot["Improvement"] = pivot["Detrended"] - pivot["Raw"]

# print(pivot)
# Compute average improvement and std per Method and Metric (i.e. task)
summary = pivot.groupby(["Method", "Metric"])["Improvement"].agg(['mean', 'std']).reset_index()
# print(summary)
summary["formatted"] = summary.apply(lambda x: f"{x['mean']:.3f} $\\pm$ {x['std']:.3f}", axis=1)

# summary
# Pivot to get desired LaTeX format
latex_table = summary.pivot(index="Method", columns="Metric", values="formatted").fillna("—")

method_order = ["MSP", "PPL", "MTE", "MCSE", "MCNSE", "LSRL"]

# Reindex before converting to LaTeX
latex_table = latex_table.reindex(method_order)

metric_order = ["Comet WMT22", "XComet XXL", "MetricX-XXL", "Align Score", "Acc"]

# Reindex columns after pivot
latex_table = latex_table.reindex(columns=metric_order)

# Convert to LaTeX
latex_code = latex_table.to_latex(escape=False)
print(latex_code)


\begin{tabular}{llllll}
\toprule
Metric & Comet WMT22 & XComet XXL & MetricX-XXL & Align Score & Acc \\
Method &  &  &  &  &  \\
\midrule
MSP & 0.090 $\pm$ 0.077 & 0.085 $\pm$ 0.103 & 0.181 $\pm$ 0.065 & 0.028 $\pm$ 0.002 & -0.001 $\pm$ 0.008 \\
PPL & 0.052 $\pm$ 0.045 & 0.051 $\pm$ 0.055 & 0.019 $\pm$ 0.024 & 0.007 $\pm$ 0.016 & 0.092 $\pm$ 0.025 \\
MTE & 0.081 $\pm$ 0.050 & 0.069 $\pm$ 0.067 & 0.028 $\pm$ 0.030 & 0.008 $\pm$ 0.021 & 0.084 $\pm$ 0.033 \\
MCSE & 0.067 $\pm$ 0.076 & 0.071 $\pm$ 0.103 & 0.162 $\pm$ 0.068 & 0.020 $\pm$ 0.014 & 0.003 $\pm$ 0.004 \\
MCNSE & 0.018 $\pm$ 0.027 & 0.017 $\pm$ 0.031 & 0.002 $\pm$ 0.014 & 0.011 $\pm$ 0.007 & 0.017 $\pm$ 0.002 \\
LSRL & -0.003 $\pm$ 0.030 & 0.006 $\pm$ 0.033 & 0.000 $\pm$ 0.012 & 0.026 $\pm$ 0.022 & -0.003 $\pm$ 0.001 \\
\bottomrule
\end{tabular}

